In [35]:
import pandas as pd
# !pip install gensim
import numpy as np
import gensim
import os
import string
import re
import pprint
from gensim import corpora, models, similarities
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
import nltk
from nltk import word_tokenize, tokenize
nltk.download('punkt')

%matplotlib inline
from matplotlib import pyplot as plt
#!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import seaborn as sns
import datetime
from collections import Counter
from tqdm import tqdm
tqdm.pandas(desc='progress-bar')
from gensim.models.doc2vec import LabeledSentence
import operator

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
trump_pd = pd.read_csv('../data/hashtag_donaldtrump.csv', engine='python')
trump_df = trump_pd[trump_pd['country'] == 'United States of America']
trump_df.shape

(178897, 21)

In [3]:
biden_pd = pd.read_csv('../data/hashtag_joebiden.csv', engine='python')
biden_df = biden_pd[biden_pd['country'] == 'United States of America']
biden_df.shape

(153595, 21)

In [4]:
def clean_dates(df, date):    
    df['created_datetime'] = pd.to_datetime(df['created_at'])
    return df[df['created_datetime'] < date]

In [5]:
trump_df = clean_dates(trump_df, datetime.datetime(2020,11,3))
biden_df = clean_dates(biden_df, datetime.datetime(2020,11,3))

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [45]:
def generate_polarity_scores(df, is_biden):
    tweets = df['tweet']
    punct_re = r'[^(\w)#!?@]'
    analyzer = SentimentIntensityAnalyzer()
    clean_tweets = tweets.str.replace(punct_re, ' ', regex=True).to_list()

    #polarity_list = []
    voting = []
    for tweet in clean_tweets:
        vs = analyzer.polarity_scores(tweet)
        #vs['tweet'] = tweet
        #polarity_list.append(vs)
        difference = (vs['pos'] - vs['neg'])
        if not is_biden:
            difference *= -1
        if difference > 0.05:
            voting.append("biden")
        elif difference < -0.05:
            voting.append("trump")
        else:
            voting.append("neutral")
    
    return voting
    
#     return polarity_list
#     new_df = pd.DataFrame(polarity_list)
#     new_df['state'] = df['state']
#     new_df = new_df[['tweet', 'state', 'compound', 'pos', 'neg', 'neu']].sort_values(by=['compound'], ascending=False)
#     return new_df

In [46]:
trump_df['voting'] = generate_polarity_scores(trump_df, False)
biden_df['voting'] = generate_polarity_scores(biden_df, True)
trump_df.groupby("voting").count()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,lat,long,city,country,continent,state,state_code,collected_at,created_datetime,hashtag
voting,,,,,,,,,,,,,,,,,,,,,
biden,35663,35663,35663,35663,35663,35661,35663,35663,35663,33906,...,35663,35663,19611,35663,35663,35658,35658,35663,35663,35663
neutral,47276,47276,47276,47276,47276,47271,47276,47275,47276,44681,...,47276,47276,26503,47276,47276,47276,47276,47276,47276,47276
trump,33351,33351,33351,33351,33351,33348,33351,33351,33351,31659,...,33351,33351,18123,33351,33351,33347,33347,33351,33351,33351


In [6]:
trump_df['hashtag'] = 'trump'
biden_df['hashtag'] = 'biden'
full_df = pd.concat([trump_df, biden_df], ignore_index=True)
full_df = full_df[['tweet', 'user_screen_name', 'user_description', 'likes', 'retweet_count', 'state_code', 'hashtag']]
full_df.head(10)

,tweet,user_screen_name,user_description,likes,retweet_count,state_code,hashtag
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,elsollatinonews,ðŸŒ� Noticias de interÃ©s para latinos de la c...,0.0,0.0,FL,trump
1,"#Trump: As a student I used to hear for years,...",snarke,"Will mock for food! Freelance writer, blogger,...",2.0,1.0,OR,trump
2,You get a tie! And you get a tie! #Trump â€˜s ...,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",4.0,3.0,DC,trump
3,@CLady62 Her 15 minutes were over long time ag...,FarrisFlagg,#BidenHarris2020 #JoeBiden2020 #KamalaHarrisFo...,2.0,0.0,CA,trump
4,@DeeviousDenise @realDonaldTrump @nypost There...,sm_gulledge,"Patriot, Wife, â€œShaken not Stirredâ€� Mom of...",0.0,0.0,OH,trump
5,One of the single most effective remedies to e...,jamieo33,"Don't know what I am. Can lean left and right,...",0.0,0.0,PA,trump
6,"In 2020, #NYPost is being #censorship #CENSORE...",changeillinois,"Illinois, home of Lincoln and Reagan, used to ...",0.0,0.0,IL,trump
7,#Trump #PresidentTrump #Trump2020LandslideVict...,Anchorman_USA,"I'm kind of a Big Deal, People know me! I driv...",3.0,5.0,CA,trump
8,"@cnnbrk #Trump owes #RicardoAguirre $730,000 t...",MoClarker,Media Maven/Scientist/Fan O Fauci,3.0,2.0,MI,trump
9,"#Trump: Nobody likes to tell you this, but som...",snarke,"Will mock for food! Freelance writer, blogger,...",1.0,1.0,OR,trump


In [7]:
def remove_mentions(df):
    
    pattern = r"@[\w]*"
    tweets = df['tweet']
    
    clean_tweets = []
    for tweet in tweets:
        r = re.findall(pattern, tweet)
        for i in r:
            tweet = re.sub(i, '', tweet)
        clean_tweets.append(tweet)
            
    return clean_tweets

In [8]:
full_df['tweet'] = remove_mentions(full_df)
full_df['tweet'] = full_df['tweet'].str.replace("[^a-zA-Z#]", " ")

full_df.head()
#finished removing punctuations and mentions

,tweet,user_screen_name,user_description,likes,retweet_count,state_code,hashtag
0,#Elecciones En #Florida #JoeBiden dice ...,elsollatinonews,ðŸŒ� Noticias de interÃ©s para latinos de la c...,0.0,0.0,FL,trump
1,#Trump As a student I used to hear for years ...,snarke,"Will mock for food! Freelance writer, blogger,...",2.0,1.0,OR,trump
2,You get a tie And you get a tie #Trump s ...,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",4.0,3.0,DC,trump
3,Her minutes were over long time ago Omaro...,FarrisFlagg,#BidenHarris2020 #JoeBiden2020 #KamalaHarrisFo...,2.0,0.0,CA,trump
4,There won t be many of them Unless you ...,sm_gulledge,"Patriot, Wife, â€œShaken not Stirredâ€� Mom of...",0.0,0.0,OH,trump


In [9]:
full_df['tweet'] = full_df['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
full_df.head()
#got rid of short words to reduce noise

,tweet,user_screen_name,user_description,likes,retweet_count,state_code,hashtag
0,#Elecciones #Florida #JoeBiden dice #DonaldTru...,elsollatinonews,ðŸŒ� Noticias de interÃ©s para latinos de la c...,0.0,0.0,FL,trump
1,#Trump student used hear years years heard Chi...,snarke,"Will mock for food! Freelance writer, blogger,...",2.0,1.0,OR,trump
2,#Trump rally #Iowa https jJalUUmh,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",4.0,3.0,DC,trump
3,minutes were over long time Omarosa never repr...,FarrisFlagg,#BidenHarris2020 #JoeBiden2020 #KamalaHarrisFo...,2.0,0.0,CA,trump
4,There many them Unless have been voting more t...,sm_gulledge,"Patriot, Wife, â€œShaken not Stirredâ€� Mom of...",0.0,0.0,OH,trump


In [14]:
tweet_tokens = full_df['tweet'].apply(lambda x: x.split())
print(tweet_tokens[:5])

0    [#Elecciones, #Florida, #JoeBiden, dice, #Dona...
1    [#Trump, student, used, hear, years, years, he...
2              [#Trump, rally, #Iowa, https, jJalUUmh]
3    [minutes, were, over, long, time, Omarosa, nev...
4    [There, many, them, Unless, have, been, voting...
Name: tweet, dtype: object


In [15]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
tweet_tokens = tweet_tokens.apply(lambda x: [stemmer.stem(i) for i in x])
tweet_tokens[:5]
#normalizing tweets using porter stemmer

0    [#eleccion, #florida, #joebiden, dice, #donald...
1    [#trump, student, use, hear, year, year, heard...
2               [#trump, ralli, #iowa, http, jjaluumh]
3    [minut, were, over, long, time, omarosa, never...
4    [there, mani, them, unless, have, been, vote, ...
Name: tweet, dtype: object

In [18]:
full_model = gensim.models.Word2Vec(tweet_tokens, min_count=32, size=100, window=5, sg=1, workers=32, seed=0)
full_model.train(tweet_tokens, total_examples=len(full_df['tweet']), epochs=30)

(72242602, 99509100)

In [23]:
full_model.most_similar(positive='biden')

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('trump', 0.606967568397522),
 ('hunter', 0.5977370738983154),
 ('#biden', 0.5962669849395752),
 ('kamala', 0.5762556791305542),
 ('harri', 0.5639647245407104),
 ('#joebiden', 0.5451806783676147),
 ('hillari', 0.4962920844554901),
 ('seguir', 0.4650038778781891),
 ('email', 0.45872586965560913),
 ('clave', 0.456613153219223)]

In [19]:
for i in range(len(tweet_tokens)):
    tweet_tokens[i] = ' '.join(tweet_tokens[i])
full_df['tweet'] = tweet_tokens
full_df.head()

,tweet,user_screen_name,user_description,likes,retweet_count,state_code,hashtag
0,#eleccion #florida #joebiden dice #donaldtrump...,elsollatinonews,ðŸŒ� Noticias de interÃ©s para latinos de la c...,0.0,0.0,FL,trump
1,#trump student use hear year year heard china ...,snarke,"Will mock for food! Freelance writer, blogger,...",2.0,1.0,OR,trump
2,#trump ralli #iowa http jjaluumh,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",4.0,3.0,DC,trump
3,minut were over long time omarosa never repres...,FarrisFlagg,#BidenHarris2020 #JoeBiden2020 #KamalaHarrisFo...,2.0,0.0,CA,trump
4,there mani them unless have been vote more tha...,sm_gulledge,"Patriot, Wife, â€œShaken not Stirredâ€� Mom of...",0.0,0.0,OH,trump


In [26]:
def add_label(twt):
    output = []
    for i, s in zip(twt.index, twt):
        output.append(LabeledSentence(s, ['tweet_', str(i)]))
    return output

In [28]:
labeled_tweets = add_label(tweet_tokens)
labeled_tweets[:5]

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  after removing the cwd from sys.path.


[LabeledSentence(words='#eleccion #florida #joebiden dice #donaldtrump solo preocupa mismo crata anfitri encuentro elector #pembrokepin #miramar clic http qhiwpiuxst #elsollatino #yobrilloconelsol http flcbwf', tags=['tweet_', '0']),
 LabeledSentence(words='#trump student use hear year year heard china have they know mani have ask them mani have they said know have million like million what', tags=['tweet_', '1']),
 LabeledSentence(words='#trump ralli #iowa http jjaluumh', tags=['tweet_', '2']),
 LabeledSentence(words='minut were over long time omarosa never repres black commun #thereidout cri #trump beg', tags=['tweet_', '3']),
 LabeledSentence(words='there mani them unless have been vote more than onc again prevail most corrupt presid ever dark light your lie come through they wouldn last forev #trump', tags=['tweet_', '4'])]

In [30]:
full_doc_model = gensim.models.Doc2Vec(dm = 1, dm_mean=1, vector_size=100, window=5, min_count=5, workers=32, alpha=0.1, seed=0)
full_doc_model.build_vocab([i for i in tqdm(labeled_tweets)])
full_doc_model.train(labeled_tweets, total_examples=len(full_df['tweet']), epochs=20)

100%|█████████████████████████████████████████████████████████████████████| 205689/205689 [00:00<00:00, 3601879.57it/s]


In [31]:
docvec_arrays = np.zeros((len(tweet_tokens), 100))
for i in range(len(full_df)):
    docvec_arrays[i,:] = full_doc_model.docvecs[i].reshape((1, 100))
    
docvec_df = pd.DataFrame(docvec_arrays)
docvec_df.shape

(205689, 100)